In [1]:
import requests
import re
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import time
import datetime
import sys
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob
import csv
from configparser import ConfigParser

In [2]:
from configparser import ConfigParser    
config = ConfigParser()

config_file = os.path.join(os.path.dirname('__file__'), 'config.ini')
config.read(config_file)
default = config['user.data']
user=default['username']
passwd=default['password']
trainQ=default['trainQtr']
testQ=default['testQtr']

print("USERNAME=" + user)
print("PASSWORD=" + passwd)
print("TRAINQUARTER=" + (trainQ))
print("TESTQUARTER=" + (testQ))

USERNAME=yadav.ank@Husky.neu.edu
PASSWORD=pZF<P@U_
TRAINQUARTER=Q12005
TESTQUARTER=Q22005


In [10]:
def  getTrainData(trainQ):    
    print("Starting train data download")
    downloadPath='./HistoricalInputFiles/historical_data1_time_'+trainQ+'.txt'

    c_size = 2500000
    df = pd.read_csv('./head.txt', sep="|",
                     names=['LOAN_SEQ_NO', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'CURR_LOAN_DEL_STATUS',
                            'LOAN_AGE', 'REM_MTH_LEGAL_MATURITY', 'REPURCHASE_FLAG', 'MODIFICATION_FLAG',
                            'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFF_DATE', 'CURRENT_INTEREST_DATE',
                            'CURRENT_DEFERRED_UPB', 'DUE_DATE_LAST_PAID_INST', 'MI_RECOVERIES',
                            'NET_SALES_PROCEEDS',
                            'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAIN_PRES_COSTS',
                            'TAXES_INSURANCE', 'MISC_EXPENSES', 'ACTUAL_LOSS', 'MODIFICATION_COST', 'STEP_MOD_FLAG',
                            'DEFERRED_PAYMENT_MODI', 'EST_LOAN_TO_VALUE'],
                     skipinitialspace=True, error_bad_lines=False, index_col=False, dtype='unicode')

    for gm_chunk in pd.read_csv(downloadPath, sep="|",
                                names=['LOAN_SEQ_NO', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB',
                                       'CURR_LOAN_DEL_STATUS',
                                       'LOAN_AGE', 'REM_MTH_LEGAL_MATURITY', 'REPURCHASE_FLAG', 'MODIFICATION_FLAG',
                                       'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFF_DATE', 'CURRENT_INTEREST_DATE',
                                       'CURRENT_DEFERRED_UPB', 'DUE_DATE_LAST_PAID_INST', 'MI_RECOVERIES',
                                       'NET_SALES_PROCEEDS',
                                       'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAIN_PRES_COSTS',
                                       'TAXES_INSURANCE', 'MISC_EXPENSES', 'ACTUAL_LOSS', 'MODIFICATION_COST',
                                       'STEP_MOD_FLAG',
                                       'DEFERRED_PAYMENT_MODI', 'EST_LOAN_TO_VALUE'],
                                skipinitialspace=True, error_bad_lines=False, index_col=False, dtype='unicode',
                                chunksize=c_size):
        frames = [df, gm_chunk]
        df = pd.concat(frames)
        print(df.shape)
        break

    df.head()
   # df.describe()
    print("done train data download")

    df = df.drop(columns=['REPURCHASE_FLAG', 'MODIFICATION_FLAG', 'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFF_DATE',
                          'DUE_DATE_LAST_PAID_INST', 'MI_RECOVERIES',
                          'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAIN_PRES_COSTS',
                          'TAXES_INSURANCE',
                          'MISC_EXPENSES', 'ACTUAL_LOSS', 'MODIFICATION_COST', 'STEP_MOD_FLAG', 'DEFERRED_PAYMENT_MODI',
                          'EST_LOAN_TO_VALUE'])
    print("Starting train data cleansing")

    df.CURRENT_ACTUAL_UPB = df.CURRENT_ACTUAL_UPB.astype('float64')
    df.CURRENT_DEFERRED_UPB = df.CURRENT_DEFERRED_UPB.astype('float64')
    df.CURRENT_INTEREST_DATE = df.CURRENT_INTEREST_DATE.astype('float64')
    df[['MONTHLY_REPORTING_PERIOD', 'LOAN_AGE', 'REM_MTH_LEGAL_MATURITY']] = df[
        ['MONTHLY_REPORTING_PERIOD', 'LOAN_AGE', 'REM_MTH_LEGAL_MATURITY']].astype('int64')
    df[['LOAN_SEQ_NO', 'CURR_LOAN_DEL_STATUS']] = df[['LOAN_SEQ_NO', 'CURR_LOAN_DEL_STATUS']].astype('str')
    df['CURR_LOAN_DEL_STATUS'] = [999 if x == 'R' else x for x in (df['CURR_LOAN_DEL_STATUS'].apply(lambda x: x))]
    df['CURR_LOAN_DEL_STATUS'] = [0 if x == 'XX' else x for x in (df['CURR_LOAN_DEL_STATUS'].apply(lambda x: x))]

    df['YEAR'] = ['19' + x if x == '99' else '20' + x for x in (df['LOAN_SEQ_NO'].apply(lambda x: x[2:4]))]
    df['QUARTER'] = df['LOAN_SEQ_NO'].apply(lambda x: x[4:6])

    print("done train data cleanising")

    df[['CURR_LOAN_DEL_STATUS']] = df[['CURR_LOAN_DEL_STATUS']].astype('int64')
    df['DELINQUENT'] = (df.CURR_LOAN_DEL_STATUS > 0).astype(int)
    # df.drop('max_curr_ln_delin_status', axis = 1,inplace=True)
    # df.drop('CURR_LOAN_DEL_STATUS', axis = 1,inplace=True)

    df.drop('CURR_LOAN_DEL_STATUS', axis=1, inplace=True)

    print("exporting train data to csv")

    df.to_csv('trainData_' + trainQ + '.csv')
    print("train data downloaded at- "+ 'trainData_' + trainQ + '.csv')

In [7]:
def getTestData(testQ):
    print("Starting test data download")
    downloadPath = './HistoricalInputFiles/historical_data1_time_' + testQ + '.txt'

    c_size = 2500000
    test_df = pd.read_csv('./head.txt', sep="|",
                     names=['LOAN_SEQ_NO', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'CURR_LOAN_DEL_STATUS',
                            'LOAN_AGE', 'REM_MTH_LEGAL_MATURITY', 'REPURCHASE_FLAG', 'MODIFICATION_FLAG',
                            'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFF_DATE', 'CURRENT_INTEREST_DATE',
                            'CURRENT_DEFERRED_UPB', 'DUE_DATE_LAST_PAID_INST', 'MI_RECOVERIES',
                            'NET_SALES_PROCEEDS',
                            'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAIN_PRES_COSTS',
                            'TAXES_INSURANCE', 'MISC_EXPENSES', 'ACTUAL_LOSS', 'MODIFICATION_COST', 'STEP_MOD_FLAG',
                            'DEFERRED_PAYMENT_MODI', 'EST_LOAN_TO_VALUE'],
                     skipinitialspace=True, error_bad_lines=False, index_col=False, dtype='unicode')

    for gm_chunk in pd.read_csv(downloadPath, sep="|",
                                names=['LOAN_SEQ_NO', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB',
                                       'CURR_LOAN_DEL_STATUS',
                                       'LOAN_AGE', 'REM_MTH_LEGAL_MATURITY', 'REPURCHASE_FLAG', 'MODIFICATION_FLAG',
                                       'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFF_DATE', 'CURRENT_INTEREST_DATE',
                                       'CURRENT_DEFERRED_UPB', 'DUE_DATE_LAST_PAID_INST', 'MI_RECOVERIES',
                                       'NET_SALES_PROCEEDS',
                                       'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAIN_PRES_COSTS',
                                       'TAXES_INSURANCE', 'MISC_EXPENSES', 'ACTUAL_LOSS', 'MODIFICATION_COST',
                                       'STEP_MOD_FLAG',
                                       'DEFERRED_PAYMENT_MODI', 'EST_LOAN_TO_VALUE'],
                                skipinitialspace=True, error_bad_lines=False, index_col=False, dtype='unicode',
                                chunksize=c_size):
        frames = [test_df, gm_chunk]
        test_df = pd.concat(frames)
        print(test_df.shape)
        break


    #test_df.head()

    print("done test data download")

    test_df = test_df.drop(
        columns=['REPURCHASE_FLAG', 'MODIFICATION_FLAG', 'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFF_DATE',
                 'DUE_DATE_LAST_PAID_INST', 'MI_RECOVERIES',
                 'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAIN_PRES_COSTS',
                 'TAXES_INSURANCE',
                 'MISC_EXPENSES', 'ACTUAL_LOSS', 'MODIFICATION_COST', 'STEP_MOD_FLAG', 'DEFERRED_PAYMENT_MODI',
                 'EST_LOAN_TO_VALUE'])
    print("Starting test data cleansing")

    test_df.CURRENT_ACTUAL_UPB = test_df.CURRENT_ACTUAL_UPB.astype('float64')
    test_df.CURRENT_DEFERRED_UPB = test_df.CURRENT_DEFERRED_UPB.astype('float64')
    test_df.CURRENT_INTEREST_DATE = test_df.CURRENT_INTEREST_DATE.astype('float64')
    test_df[['MONTHLY_REPORTING_PERIOD', 'LOAN_AGE', 'REM_MTH_LEGAL_MATURITY']] = test_df[
        ['MONTHLY_REPORTING_PERIOD', 'LOAN_AGE',
         'REM_MTH_LEGAL_MATURITY']].astype('int64')

    test_df[['LOAN_SEQ_NO', 'CURR_LOAN_DEL_STATUS']] = test_df[['LOAN_SEQ_NO', 'CURR_LOAN_DEL_STATUS']].astype('str')
    test_df['CURR_LOAN_DEL_STATUS'] = [999 if x == 'R' else x for x in
                                       (test_df['CURR_LOAN_DEL_STATUS'].apply(lambda x: x))]
    test_df['CURR_LOAN_DEL_STATUS'] = [0 if x == 'XX' else x for x in
                                       (test_df['CURR_LOAN_DEL_STATUS'].apply(lambda x: x))]

    test_df['YEAR'] = ['19' + x if x == '99' else '20' + x for x in (test_df['LOAN_SEQ_NO'].apply(lambda x: x[2:4]))]
    test_df['QUARTER'] = test_df['LOAN_SEQ_NO'].apply(lambda x: x[4:6])

    print("done test data cleanising")

    test_df[['CURR_LOAN_DEL_STATUS']] = test_df[['CURR_LOAN_DEL_STATUS']].astype('int64')
    test_df['DELINQUENT'] = (test_df.CURR_LOAN_DEL_STATUS > 0).astype(int)
    # df.drop('max_curr_ln_delin_status', axis = 1,inplace=True)
    # df.drop('CURR_LOAN_DEL_STATUS', axis = 1,inplace=True)

    test_df.drop('CURR_LOAN_DEL_STATUS', axis=1, inplace=True)


    print("exporting test data to csv")

    test_df.to_csv('testData_' + testQ + '.csv')
    print("test data downloaded at- "+ 'testData_' + testQ + '.csv')

In [11]:
getTrainData(trainQ)

Starting train data download
(2500000, 26)
done train data download
Starting train data cleansing
done train data cleanising
exporting train data to csv
train data downloaded at- trainData_Q12005.csv


In [9]:
getTestData(testQ)

Starting test data download
(2500000, 26)
done test data download
Starting test data cleansing
done test data cleanising
exporting test data to csv
test data downloaded at- testData_Q22005.csv
